# **Auxilliary**

In [ ]:
# Library Imports
import sys
import pandas as pd

from dataset import dataset_generator
from bot.bot import Bot
from modelling import data_preparation as dp
from modelling import model_trainer as mt

path = "binary-options-research/"
if path not in sys.path:
    sys.path.append(path)
sys.stdout = open("stdout", 'w')

%load_ext autoreload
%autoreload 2

# **Data Generation**

In [ ]:
# Dataset Generation
dataset_generator.make(["BTCUSDT"], "1m")

# **Data Extraction**

In [ ]:
# Data Extraction
candles = pd.read_csv(f"dataset/BTCUSDT-1m-2020_2024.csv", index_col=0)
candles_training = candles[(candles['open_time'] >= '2020-01-01') & (candles['open_time'] < '2023-01-01')].reset_index(drop=True)

bot = Bot(candles_training, 0)
bot._pattern_features_service()

# **Model Training**

In [ ]:
# Save data Double Bottom
features = pd.read_csv("features/features.csv")
features_db = features[features['pattern'] == "Double Bottom"].reset_index(drop=True)
X_train, y_train, X_valid, y_valid, X_test, y_test = dp.split_data(features_db)
print("Writing data")
dp.save_data(X_train, y_train, X_valid, y_valid, X_test, y_test)

In [ ]:
# Parameter Hypertuning Double Bottom
X_train, y_train, X_valid, y_valid, X_test, y_test = dp.load_data()
mt.find_hyper_params(X_train, y_train, X_valid, y_valid)

In [ ]:
# Model Training Double Bottom
hyper = {'base_score': 7, 'colsample_bylevel': 9, 'colsample_bynode': 1, 'colsample_bytree': 9, 'gamma': 2, 'learning_rate': 6, 'max_depth': 9, 'min_child_weight': 0, 'n_estimators': 0, 'reg_alpha': 3, 'reg_lambda': 3, 'scale_pos_weight': 8, 'subsample': 3}
X_train, y_train, X_valid, y_valid, X_test, y_test  = dp.load_data()
mt.fit_model(X_train, y_train, X_valid, y_valid, X_test, y_test, hyper, "DBmodel") 

In [ ]:
# Save data Double Top
features = pd.read_csv("features/features.csv")
features_dt = features[features['pattern'] == "Double Top"].reset_index(drop=True)
X_train, y_train, X_valid, y_valid, X_test, y_test = dp.split_data(features_dt)
print("Writing data")
dp.save_data(X_train, y_train, X_valid, y_valid, X_test, y_test)

In [109]:
# Parameter Hypertuning Double Top
X_train, y_train, X_valid, y_valid, X_test, y_test = dp.load_data()
mt.find_hyper_params(X_train, y_train, X_valid, y_valid)

In [111]:
# Model Training Double Top
hyper = {'base_score': 1, 'colsample_bylevel': 11, 'colsample_bynode': 0, 'colsample_bytree': 9, 'gamma': 0, 'learning_rate': 7, 'max_depth': 3, 'min_child_weight': 0, 'n_estimators': 2, 'reg_alpha': 5, 'reg_lambda': 9, 'scale_pos_weight': 10, 'subsample': 2}
X_train, y_train, X_valid, y_valid, X_test, y_test  = dp.load_data()
mt.fit_model(X_train, y_train, X_valid, y_valid, X_test, y_test, hyper, "DTmodel") 

# **Forward Testing**

In [ ]:
# Model testing
candles = pd.read_csv(f"dataset/BTCUSDT-1m-2020_2024.csv", index_col=0)
candles_training = candles[(candles['open_time'] >= '2023-01-01')].reset_index(drop=True)

bot = Bot(candles_training, 1)
bot._pattern_features_service()